In [ ]:
import pymysql
import pymongo
import filetype
import os
import json
import numpy as np
from datetime import datetime
from sentence_transformers import SentenceTransformer

class DatabaseManager:
    def __init__(self, mysql_config, mongodb_config):
        self.mysql_config = mysql_config
        self.mongodb_config = mongodb_config
        self.mysql_conn = None
        self.mongodb_client = None
        self.mongodb_db = None
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

    def connect_mysql(self):
        try:
            self.mysql_conn = pymysql.connect(
                host=self.mysql_config['host'],
                user=self.mysql_config['user'],
                password=self.mysql_config['password'],
                database=self.mysql_config['database'],
                charset='utf8mb4',
                cursorclass=pymysql.cursors.DictCursor
            )
            print("MySQL connection established")
        except Exception as e:
            print(f"MySQL connection failed: {e}")

    def connect_mongodb(self):
        try:
            self.mongodb_client = pymongo.MongoClient(self.mongodb_config['uri'])
            self.mongodb_db = self.mongodb_client[self.mongodb_config['database']]
            print("MongoDB connection established")
        except Exception as e:
            print(f"MongoDB connection failed: {e}")

    def setup_mysql_procedures(self):
        procedures = [
            '''
            CREATE PROCEDURE IF NOT EXISTS CleanTextData(
                IN file_id INT,
                IN raw_content TEXT
            )
            BEGIN
                DECLARE cleaned_content TEXT;

                SET cleaned_content = TRIM(raw_content);
                SET cleaned_content = REPLACE(cleaned_content, '\\n', ' ');
                SET cleaned_content = REPLACE(cleaned_content, '\\r', ' ');
                SET cleaned_content = REPLACE(cleaned_content, '\\t', ' ');

                UPDATE structured_files
                SET content = cleaned_content,
                    processed_at = NOW()
                WHERE id = file_id;
            END
            ''',

            '''
            CREATE PROCEDURE IF NOT EXISTS CleanNumericData(
                IN file_id INT,
                IN raw_data JSON
            )
            BEGIN
                DECLARE cleaned_data JSON;

                SET cleaned_data = JSON_REMOVE(raw_data, '$.null_values');

                UPDATE structured_files
                SET content = cleaned_data,
                    processed_at = NOW()
                WHERE id = file_id;
            END
            ''',

            '''
            CREATE PROCEDURE IF NOT EXISTS ValidateFileData(
                IN file_id INT
            )
            BEGIN
                DECLARE file_size INT DEFAULT 0;

                SELECT LENGTH(content) INTO file_size
                FROM structured_files
                WHERE id = file_id;

                IF file_size = 0 THEN
                    UPDATE structured_files
                    SET status = 'invalid'
                    WHERE id = file_id;
                ELSE
                    UPDATE structured_files
                    SET status = 'valid'
                    WHERE id = file_id;
                END IF;
            END
            '''
        ]

        if self.mysql_conn:
            cursor = self.mysql_conn.cursor()
            for procedure in procedures:
                cursor.execute(procedure)
            self.mysql_conn.commit()
            cursor.close()
            print("MySQL stored procedures created")

    def create_mysql_tables(self):
        tables = [
            '''
            CREATE TABLE IF NOT EXISTS structured_files (
                id INT AUTO_INCREMENT PRIMARY KEY,
                filename VARCHAR(255) NOT NULL,
                file_type VARCHAR(50) NOT NULL,
                file_size INT NOT NULL,
                content LONGTEXT,
                status ENUM('pending', 'processing', 'valid', 'invalid') DEFAULT 'pending',
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                processed_at TIMESTAMP NULL
            )
            ''',

            '''
            CREATE TABLE IF NOT EXISTS file_metadata (
                id INT AUTO_INCREMENT PRIMARY KEY,
                file_id INT,
                metadata_key VARCHAR(100),
                metadata_value TEXT,
                FOREIGN KEY (file_id) REFERENCES structured_files(id)
            )
            '''
        ]

        if self.mysql_conn:
            cursor = self.mysql_conn.cursor()
            for table in tables:
                cursor.execute(table)
            self.mysql_conn.commit()
            cursor.close()
            print("MySQL tables created")

    def create_mongodb_collections(self):
        if self.mongodb_db:
            self.mongodb_db.create_collection('unstructured_files')
            self.mongodb_db.unstructured_files.create_index([("filename", 1)])
            self.mongodb_db.unstructured_files.create_index([("file_type", 1)])
            print("MongoDB collections and indexes created")

    def close_connections(self):
        if self.mysql_conn:
            self.mysql_conn.close()
        if self.mongodb_client:
            self.mongodb_client.close()

class FileAnalyzer:
    def __init__(self):
        self.structured_types = [
            'text/csv', 'application/json', 'text/xml', 'application/xml',
            'text/plain', 'application/vnd.ms-excel',
            'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
        ]

    def analyze_file(self, file_path):
        file_info = {
            'filename': os.path.basename(file_path),
            'file_size': os.path.getsize(file_path),
            'file_type': None,
            'mime_type': None,
            'is_structured': False
        }

        kind = filetype.guess(file_path)
        if kind:
            file_info['file_type'] = kind.extension
            file_info['mime_type'] = kind.mime
        else:
            file_info['file_type'] = 'unknown'
            file_info['mime_type'] = 'application/octet-stream'

        file_info['is_structured'] = file_info['mime_type'] in self.structured_types

        return file_info

class FileProcessor:
    def __init__(self, db_manager):
        self.db_manager = db_manager
        self.analyzer = FileAnalyzer()

    def read_file_content(self, file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        except UnicodeDecodeError:
            with open(file_path, 'rb') as f:
                return f.read().decode('utf-8', errors='ignore')
        except Exception as e:
            return str(e)

    def process_single_file(self, file_path):
        file_info = self.analyzer.analyze_file(file_path)
        content = self.read_file_content(file_path)

        if file_info['is_structured']:
            return self.store_in_mysql(file_info, content)
        else:
            return self.store_in_mongodb(file_info, content)

    def store_in_mysql(self, file_info, content):
        if not self.db_manager.mysql_conn:
            return {'error': 'MySQL connection not available'}

        try:
            cursor = self.db_manager.mysql_conn.cursor()

            insert_query = '''
            INSERT INTO structured_files (filename, file_type, file_size, content, status)
            VALUES (%s, %s, %s, %s, 'pending')
            '''

            cursor.execute(insert_query, (
                file_info['filename'],
                file_info['file_type'],
                file_info['file_size'],
                content
            ))

            file_id = cursor.lastrowid

            if file_info['file_type'] in ['txt', 'csv']:
                cursor.callproc('CleanTextData', [file_id, content])
            elif file_info['file_type'] == 'json':
                cursor.callproc('CleanNumericData', [file_id, content])

            cursor.callproc('ValidateFileData', [file_id])

            self.db_manager.mysql_conn.commit()
            cursor.close()

            return {'success': True, 'storage': 'MySQL', 'file_id': file_id}

        except Exception as e:
            return {'error': f'MySQL storage failed: {e}'}

    def store_in_mongodb(self, file_info, content):
        if not self.db_manager.mongodb_db:
            return {'error': 'MongoDB connection not available'}

        try:
            embeddings = self.db_manager.embedding_model.encode([content])[0].tolist()

            document = {
                'filename': file_info['filename'],
                'file_type': file_info['file_type'],
                'file_size': file_info['file_size'],
                'content': content,
                'embeddings': embeddings,
                'created_at': datetime.now()
            }

            result = self.db_manager.mongodb_db.unstructured_files.insert_one(document)

            return {'success': True, 'storage': 'MongoDB', 'document_id': str(result.inserted_id)}

        except Exception as e:
            return {'error': f'MongoDB storage failed: {e}'}

    def process_multiple_files(self, file_paths):
        results = []
        for file_path in file_paths:
            if os.path.exists(file_path):
                result = self.process_single_file(file_path)
                result['file_path'] = file_path
                results.append(result)
            else:
                results.append({'error': f'File not found: {file_path}', 'file_path': file_path})
        return results

class VectorSearchEngine:
    def __init__(self, db_manager):
        self.db_manager = db_manager

    def search_similar_content(self, query_text, limit=5):
        if not self.db_manager.mongodb_db:
            return {'error': 'MongoDB connection not available'}

        try:
            query_embedding = self.db_manager.embedding_model.encode([query_text])[0].tolist()

            documents = list(self.db_manager.mongodb_db.unstructured_files.find())

            similarities = []
            for doc in documents:
                if 'embeddings' in doc:
                    similarity = np.dot(query_embedding, doc['embeddings']) / (
                        np.linalg.norm(query_embedding) * np.linalg.norm(doc['embeddings'])
                    )
                    similarities.append({
                        'document': doc,
                        'similarity': similarity
                    })

            similarities.sort(key=lambda x: x['similarity'], reverse=True)

            results = []
            for i, item in enumerate(similarities[:limit]):
                doc = item['document']
                results.append({
                    '_id': str(doc['_id']),
                    'filename': doc['filename'],
                    'file_type': doc['file_type'],
                    'content': doc['content'][:500],
                    'similarity_score': item['similarity']
                })

            return {'success': True, 'results': results}

        except Exception as e:
            return {'error': f'Vector search failed: {e}'}

class FileStorageInterface:
    def __init__(self, mysql_config, mongodb_config):
        self.db_manager = DatabaseManager(mysql_config, mongodb_config)
        self.processor = FileProcessor(self.db_manager)
        self.search_engine = VectorSearchEngine(self.db_manager)

    def initialize(self):
        self.db_manager.connect_mysql()
        self.db_manager.connect_mongodb()
        self.db_manager.create_mysql_tables()
        self.db_manager.create_mongodb_collections()
        self.db_manager.setup_mysql_procedures()

    def upload_files(self, file_paths):
        if isinstance(file_paths, str):
            file_paths = [file_paths]
        return self.processor.process_multiple_files(file_paths)

    def search_files(self, query, limit=5):
        return self.search_engine.search_similar_content(query, limit)

    def close(self):
        self.db_manager.close_connections()

if __name__ == "__main__":
    mysql_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '####',
        'database': 'file_storage'
    }

    mongodb_config = {
        'uri': 'mongodb://localhost:27017/',
        'database': 'file_storage'
    }

    storage_system = FileStorageInterface(mysql_config, mongodb_config)
    storage_system.initialize()

    file_paths = ['document.txt', 'data.csv', 'image.jpg']
    results = storage_system.upload_files(file_paths)
    print("Upload results:", results)

    search_results = storage_system.search_files("machine learning algorithms")
    print("Search results:", search_results)

    storage_system.close()
